# Project 8: CNN

## Instructions

### Description

In this project you will be learning about Convolutional Neural Networks (CNN) and messing with the parameters to see how they can change the output of the CNN. You will start with some questions about Tensorflow, the python package used for CNN! Good luck!

### Grading

For grading purposes, we will clear all outputs from all your cells and then run them all from the top.  Please test your notebook in the same fashion before turning it in.

### Submitting Your Solution

To submit your notebook, first clear all the cells (this won't matter too much this time, but for larger data sets in the future, it will make the file smaller).  Then use the File->Download As->Notebook to obtain the notebook file.  Finally, submit the notebook file on Canvas.


## 40 points

### Instructions
- The **"MNIST" data set** is composed of 28x28 pixel, black and white images of handwritten digits (0-9). It is commonly used to demonstrate the training and testing of CNNs.
- **We provide you with framework code** to build a CNN of roughly suitable size for the MNIST digit recognition task.


- **Your tasks**:
    1. Read the linked article and answer the questions under the "What is Tensorflow" section
    2. **Execute the train/test code** and observe the results of the gradient descent training.
    3. Complete all 4 modifications of the code, thee modification instructions are:
         - **Modify the model architecture in the following ways**, and repeat the training and testing. Each modification should be made relative to the original network. Do not keep "adding" each modification with each new model (which would lead to a final model having all the modifications below):
           - **Modification 1**: Remove the ReLU activation functions and the max-pooling layers, thereby making the entire network a linear function.
           - **Modification 2**: Increase the size of the model by a factor of 16, roughly.
           - **Modification 3**: Convert the large convolutional model to a large non-convolutional model, with roughly the same number of parameters.
           - **Modification 4**: Train the original baseline model with a larger training data set. 
     4. MAKE SURE THAT: along the way, **answer the questions in the Jupyter notebook cells**. You can just do this with comments within the cell, for example...
   - ```## Answer 1: The training loss increased when I did x, y, and z.```

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import time

np.random.seed(0)
tf.random.set_seed(0)

import matplotlib.pyplot as plt
%matplotlib inline

# Part 1
## What is TensorFlow? (6 pts)

Before we get started with the rest of the project, let's talk about what tensorflow is. You are required to read this article https://www.guru99.com/what-is-tensorflow.html, and answer the following questions:

1. What are the three parts of the TensorFlow Architecture?

    The three parts of the TensorFlow archiecture (as given in the article) are (1) preprocessing the data, (2) building the model, and (3) training/using the model. 


2. What is a Tensor and how does it represent data?

    A tensor is like a matrix but with more than two axes. It represents data in the same way that a matrix or vector would: though storing some observation in a location indexed by a vector of length n (where n is the dimension/rank of this tensor). 


3. What is one advantage of the use of graphs in TensorFlow?

    Using graphs fits very nicely with the use of tensors as it allows all operations to be done by "connecting tensors together." Additionally, it is computationally convenient as it can be run on multiple CPUs and it is more portable than other methods. 

# Part 2
## Set TensorFlow verbosity level

In [ ]:
verbose = 2 # 0==no output, 1=accuracy/loss output, 2=progress bar output

## Create helper function to plot results of our model training

In [ ]:
def plot_results(history):
    epoch_num = np.arange(1, len(history.history['loss'])+1)

    plt.figure(figsize=(15, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epoch_num, history.history['loss'], label='training_loss')
    plt.plot(epoch_num, history.history['val_loss'], label='test_loss')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epoch_num, history.history['accuracy'], label='training_accuracy')
    plt.plot(epoch_num, history.history['val_accuracy'], label='test_accuracy')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid(True)

## Load the MNIST data

### You will not need to re-run or re-use (cut/paste) the three code cells below.

The full dataset has 60,000 training images and 10,000 test images.

To accelerate training on jupyterhub (with lots of users) we will only  
use a subset of the training set. This will also let us explore some of the  
hazards of training a neural net without a very large set of samples.

In [ ]:
# Load data
(images_train, labels_train), (images_test, labels_test) = mnist.load_data()

# Use a subset of the full training and test sets for actual training and testing,
# to accelerate training, and demonstrate possible pitfalls of smaller training data sets.

n_train = 1000
images_train = images_train[0:n_train,:,:]
labels_train = labels_train[0:n_train]

n_test = 1000
images_test = images_test[0:n_test,:,:]
labels_test = labels_test[0:n_train]

## Let's graph a few of the MNIST digits, to confirm that they look as expected

Using plt.imshow(), graph the first sixteen numbers in the training images.

In [ ]:
## You will not need to run this cell more than once, or cut/paste it elsewhere
plt.figure(figsize=(8*2, 2*2))
for i in range(16):
    plt.subplot(2,8,i+1)
    plt.imshow(images_train[i,:,:], cmap='gray')

In [ ]:
# Create TensorFlow Dataset objects to hold train and test data.
images_train = images_train/255 
images_train = np.expand_dims(images_train, axis=3) # TensorFlow expects a channel dimension
images_train = tf.cast(images_train, tf.float32)
labels_train = tf.cast(labels_train, tf.float32)
dataset_train = tf.data.Dataset.from_tensor_slices((images_train, labels_train))

images_test = images_test/255
images_test = np.expand_dims(images_test, axis=3) # TensorFlow expects a channel dimension
images_test = tf.cast(images_test, tf.float32)
labels_test = tf.cast(labels_test, tf.float32)
dataset_test = tf.data.Dataset.from_tensor_slices((images_test, labels_test))

batch_size = 50

dataset_train = dataset_train.cache()
dataset_train = dataset_train.shuffle(n_train)
dataset_train = dataset_train.batch(batch_size)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

dataset_test = dataset_test.cache()
dataset_test = dataset_test.batch(batch_size)
dataset_test = dataset_test.cache()
dataset_test = dataset_test.prefetch(tf.data.experimental.AUTOTUNE)

## Construct, compile, and train the baseline model. (4 pts)


The code will plot the loss and accuracy scores that were collected during training. Remember that training uses gradient descent, so the model parameters are slowly updated as the model gets a closer and closer fit to the data. TensorFlow records the scores after each "epoch"--the number of iterations at which all samples in the training set have been used once, in the gradient descent process.

We are now going to build teh model. In tensorflow, we build the model by first defining all the layers that the model is going to have, then we compile the model. Finally, we can train the model. We are going to define the model first. The rest is taken care of for you. 

We do this by calling `tf.keras.models.Sequential()` and inputting the list of layers, in order, as the parameter. Our baseline model is going to include two convalutional layers with pooling layers after each one, then we are going to have one dense layer with 64 neurons. It will then conclude with a dense layer of 10 that is our output layer. (Hint, you need to reduce the dimensionality before the data enters the neural network part of the model!)

The following are names of layers that may be useful: 

  `tf.keras.layers.Conv2D()`

  `tf.keras.layers.MaxPool2D()`

  `tf.keras.layers.Flatten()`

  `tf.keras.layers.Dense()`

For our model, we want to make sure that our convolutional layers have a kernel size of 3x3, 4 kernels, and use the 'relu' activation. Our pooling layers are going to have a 2x2 pooling same and `padding='same'`.   Don't foget to add the relu activation to the dense layer with 64 neurons. 

More information about the layers can be found [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers)!


In [ ]:
## This is the baseline model. 

##Your Code goe here
num_kernels = 4
dense_layer_neurons = 64
kernels_size = (3,3)
pooling_size = (2,2)
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(num_kernels, kernels_size, activation = "relu"),
    tf.keras.layers.MaxPool2D(pooling_size, padding = "same"),
    
    tf.keras.layers.Conv2D(num_kernels, kernels_size, activation = "relu"),
    tf.keras.layers.MaxPool2D(pooling_size, padding = "same"),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(dense_layer_neurons, activation = "relu"),
    tf.keras.layers.Dense(10)]
)

# Do not change any arguments in the call to model.compile()
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),    
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

# Do not change any arguments in the call to model.fit()
epochs = 30
t = time.time()
history = model.fit(dataset_train,
                    epochs=epochs,
                    validation_data=dataset_test,
                    verbose=verbose)
print('Training duration: %f seconds.' % (time.time() - t))

# Plot results
plot_results(history)

## Part 2 Questions (6 pts)

### Using the baseline model, answer the questions below

## MODIFICATION 1 (6 pts)

### Copy code from cell above, that builds and trains the baseline model, and plots the results.
### Now alter the model as described below, and run the code.
Alter the model by removing the ReLU and max-pooling non-linear activations.  

In [ ]:
## MODIFICATION 1
## BELOW, PUT YOUR MODEL CONSTRUCTION, COMPILATION, AND FITTING CODE

#%tensorboard --logdir logs
# I have no clue what this is/I don't think this was ever explained. I've just commented it 
# out because it looks like it isn't supposed to be here (as none of the other modification sections have it).

num_kernels = 4
dense_layer_neurons = 64
kernels_size = (3,3)
pooling_size = (2,2)
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(num_kernels, kernels_size),    
    tf.keras.layers.Conv2D(num_kernels, kernels_size),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(dense_layer_neurons),
    tf.keras.layers.Dense(10)]
)

# Do not change any arguments in the call to model.compile()
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),    
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

# Do not change any arguments in the call to model.fit()
epochs = 30
t = time.time()
history = model.fit(dataset_train,
                    epochs=epochs,
                    validation_data=dataset_test,
                    verbose=verbose)
print('Training duration: %f seconds.' % (time.time() - t))

# Plot results
plot_results(history)

In [ ]:
## MODIFICATION 1
## QUESTION 1: During training, how did the loss (error) curves change from the baseline model to the linear (MOD 1)
##             model, for both the training and test sets? What does this imply regarding underfitting or overfitting?
## QUESTION 2: In general, did the ReLU and max-pooling non-linearities make for a better model or a worse model?

## Answer 1:
## In the training set, the loss error curve had a normal decline. In the test set, the loss error curve started 
## going up after a quick dip. This suggests that our model is overfitting the data. 
## 
## Answer 2:
## I think they made for a better model as the results for the test and training sets were more comparable than the test 
## and training results our modified model gives. 
##

## MODIFICATION 2 (6 pts)

### Copy code from the baseline model, for building, training, and plotting results.
### Now alter the model as described below, and run the code.

Make the network much larger, by:
1. Increasing the number of kernels in the convolutional layers from 4 to 64 (16x).
2. Increasing the number neurons in the dense layer from 64 to 1024 (16x).

In [ ]:
## MODIFICATION 2
## BELOW, PUT YOUR MODEL CONSTRUCTION, COMPILATION, AND FITTING CODE
num_kernels = 16
dense_layer_neurons = 1024
kernels_size = (3,3)
pooling_size = (2,2)
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(num_kernels, kernels_size, activation = "relu"),
    tf.keras.layers.MaxPool2D(pooling_size, padding = "same"),
    
    tf.keras.layers.Conv2D(num_kernels, kernels_size, activation = "relu"),
    tf.keras.layers.MaxPool2D(pooling_size, padding = "same"),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(dense_layer_neurons, activation = "relu"),
    tf.keras.layers.Dense(10)]
)

# Do not change any arguments in the call to model.compile()
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),    
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

# Do not change any arguments in the call to model.fit()
epochs = 30
t = time.time()
history = model.fit(dataset_train,
                    epochs=epochs,
                    validation_data=dataset_test,
                    verbose=verbose)
print('Training duration: %f seconds.' % (time.time() - t))

# Plot results
plot_results(history)

In [ ]:
## MODIFICATION 2
## QUESTION 1: How did the performance of the larger model (MOD 2) compare to that of the baseline model?
## QUESTION 2: Based on the training curves, does the larger model show any **clear** signs of overfitting,
##             despite the large number of parameters?

## Answer 1:
## This one is much better. Lower loss and higher accuracy for training and test sets. I don't see a downside. 
##
## Answer 2:
## Not really. We can see a little bit of an uplift in the loss curve for the test set, but it isn't dramatic/clearly 
## identifiable yet. We have no clear signs of overfitting.
##

## MODIFICATION 3 (6 pts)

### Copy code from the baseline model, for building, training, and plotting results.
### Now alter the model as described below, and run the code.

**Relative to the MOD 2 model**, remove the convolutional and max-pooling layers, replacing them with a single, new, dense layer.

The two convolutional layers have 640 and 36,928 parameters, respectively, for a total of 37,586.
For an input of 28x28 = 784 pixels (features), a dense layer with 48 neurons will have roughly 784x48 = 37,632 parameters. Thus, do the following:
1. Remove the convolutional and max-pooling layers.
2. After the Flatten() layer, add a new Dense layer with 48 neurons and a ReLU activation function.
3. Keep the (now second) Dense layer of dense_layer_neurons==1024 neurons, and the final Dense layer of 10 neurons.

In [ ]:
## MODIFICATION 3
## BELOW, PUT YOUR MODEL CONSTRUCTION, COMPILATION, AND FITTING CODE
num_kernels = 4
dense_layer_neurons = 64
kernels_size = (3,3)
pooling_size = (2,2)
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(48, activation = "relu"),
    tf.keras.layers.Dense(1024, activation = "relu"),
    tf.keras.layers.Dense(10)]
)

# Do not change any arguments in the call to model.compile()
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),    
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

# Do not change any arguments in the call to model.fit()
epochs = 30
t = time.time()
history = model.fit(dataset_train,
                    epochs=epochs,
                    validation_data=dataset_test,
                    verbose=verbose)
print('Training duration: %f seconds.' % (time.time() - t))

# Plot results
plot_results(history)

In [ ]:
## MODIFICATION 3
## QUESTION 1: How did the performance of the large non-convolutional model (MOD 3) compare to that of the
##             large convolutional model (MOD 2)?
## QUESTION 2: Speculate on why the convolutional model performs better or worse than the non-convolutional model.
##             There is a "right" answer, but we're just looking for your opinion/guess. No penalty for error.

## Answer 1:
## This one is worse. We see a diverging loss curve for the test set (implying overfitting) and our test accuracy is lower 
## than it was in MOD 2. 
## Answer 2:
## I expect that this is because our convolutional model includes operations that account for neighboring elements. I think 
## knowing how different a pixel is from its neighboring pixels is useful for me when I'm identifying a digit. It seems like
## the convolution model captures something more close to how we really see things.
##

## MODIFICATION 4 (6 pts)

## This model/data with take longer to train the the previous ones--upwards of several minutes.

### For the modification, we don't modify the model, we modify the size of the data sets. The two cells below create training and testing sets with 10,000 samples each, whereas our previous models used only 1,000 samples each.

### Copy code from the baseline model, for building, training, and plotting results.
- Place the copied code after the two cells below.  
- Run all the cells below to get the new data sets, and train the baseline model on that data.

In [ ]:
# Load data
(images_train, labels_train), (images_test, labels_test) = mnist.load_data()

# Use a subset of the full training and test sets for actual training and testing,
# to accelerate training, and demonstrate possible pitfalls of smaller training data sets.

n_train = 10000
images_train = images_train[0:n_train,:,:]
labels_train = labels_train[0:n_train]

n_test = 10000
images_test = images_test[0:n_test,:,:]
labels_test = labels_test[0:n_train]

In [ ]:
# Create TensorFlow Dataset objects to hold train and test data.
images_train = images_train/255
images_train = np.expand_dims(images_train, axis=3) # TensorFlow expects a channel dimension
images_train = tf.cast(images_train, tf.float32)
labels_train = tf.cast(labels_train, tf.float32)
dataset_train = tf.data.Dataset.from_tensor_slices((images_train, labels_train))

images_test = images_test/255
images_test = np.expand_dims(images_test, axis=3) # TensorFlow expects a channel dimension
images_test = tf.cast(images_test, tf.float32)
labels_test = tf.cast(labels_test, tf.float32)
dataset_test = tf.data.Dataset.from_tensor_slices((images_test, labels_test))

dataset_train = dataset_train.cache()
dataset_train = dataset_train.shuffle(n_train)
dataset_train = dataset_train.batch(batch_size)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

dataset_test = dataset_test.cache()
dataset_test = dataset_test.batch(batch_size)
dataset_test = dataset_test.cache()
dataset_test = dataset_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
## MODIFICATION 4
## BELOW, PUT YOUR MODEL CONSTRUCTION, COMPILATION, AND FITTING CODE
num_kernels = 4
dense_layer_neurons = 64
kernels_size = (3,3)
pooling_size = (2,2)
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(num_kernels, kernels_size, activation = "relu"),
    tf.keras.layers.MaxPool2D(pooling_size, padding = "same"),
    
    tf.keras.layers.Conv2D(num_kernels, kernels_size, activation = "relu"),
    tf.keras.layers.MaxPool2D(pooling_size, padding = "same"),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(dense_layer_neurons, activation = "relu"),
    tf.keras.layers.Dense(10)]
)

# Do not change any arguments in the call to model.compile()
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

# Do not change any arguments in the call to model.fit()
epochs = 30
t = time.time()
history = model.fit(dataset_train,
                    epochs=epochs,
                    validation_data=dataset_test,
                    verbose=verbose)
print('Training duration: %f seconds.' % (time.time() - t))

# Plot results
plot_results(history)

In [ ]:
## MODIFICATION 4
## QUESTION 1: How did the performance of the baseline model trained on the larger data set (MOD 4) compare
##             to that trained on the smaller data set?
## QUESTION 2: This is just guess on your part... how much better do you think results might be if you trained
##             the model on all 60,000 training samples (rather than 1000 or 10,000)?

## Answer 1:
## This one is one of the best yet. We have very good accuracies and our loss curves get very small. The only downside is 
## that our test loss curve looks like it is diverging a little. This is a warning sign of overfitting in our model. 
##
## Answer 2:
## If we trained on all of the samples, I think our model would probably get less accurate when identifying a new digit
## as it would overfit to the MNIST digits we are using.
##

### Questionnaire
1) How long did you spend on this assignment?
<br> I spent three hours or so on this assignment.<br>
2) What did you like about it? What did you not like about it?
<br> No comment. I did not feel particularly strongly about any aspect of it.<br>
3) Did you find any errors or is there anything you would like changed?
<br>What is "%tensorboard --logdir logs"? Why was it in the given code? <br>